### Import Libraries

In [ ]:
#%tensorflow_version 2.x
import cv2, numpy as np, matplotlib, matplotlib.pyplot as plt, numba, random, glob, os, tensorflow as tf, keras
from numba import cuda

from skimage.transform import resize
from skimage.io import imread, imshow, imsave
from contextlib import redirect_stdout
from tqdm import tqdm

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Add, Lambda, UpSampling2D
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, VGG19, InceptionResNetV2

import keras.backend as k
from keras.layers import *
from keras.layers import SpatialDropout2D, Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate,AveragePooling2D, UpSampling2D, BatchNormalization, Activation, add,Dropout,Permute,ZeroPadding2D,Add, Reshape
from keras.layers import ELU, PReLU, LeakyReLU

from keras.models import *
from keras.models import Model, model_from_json

from keras.optimizers import *
from keras import applications, optimizers, callbacks
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras import initializers

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Model 1: DeepLabv3Plus-Resnet50 and 101


In [ ]:
from tensorflow.keras.applications import ResNet50, ResNet101

""" Atrous Spatial Pyramid Pooling """
def ASPP(inputs):
    shape = inputs.shape

    y_pool = AveragePooling2D(pool_size=(shape[1], shape[2]), name='average_pooling')(inputs)
    y_pool = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(y_pool)
    y_pool = BatchNormalization(name=f'bn_1')(y_pool)
    y_pool = Activation('relu', name=f'relu_1')(y_pool)
    y_pool = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y_pool)

    y_1 = Conv2D(filters=256, kernel_size=1, dilation_rate=1, padding='same', use_bias=False)(inputs)
    y_1 = BatchNormalization()(y_1)
    y_1 = Activation('relu')(y_1)

    y_6 = Conv2D(filters=256, kernel_size=3, dilation_rate=6, padding='same', use_bias=False)(inputs)
    y_6 = BatchNormalization()(y_6)
    y_6 = Activation('relu')(y_6)

    y_12 = Conv2D(filters=256, kernel_size=3, dilation_rate=12, padding='same', use_bias=False)(inputs)
    y_12 = BatchNormalization()(y_12)
    y_12 = Activation('relu')(y_12)

    y_18 = Conv2D(filters=256, kernel_size=3, dilation_rate=18, padding='same', use_bias=False)(inputs)
    y_18 = BatchNormalization()(y_18)
    y_18 = Activation('relu')(y_18)

    y = Concatenate()([y_pool, y_1, y_6, y_12, y_18])

    y = Conv2D(filters=256, kernel_size=1, dilation_rate=1, padding='same', use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    return y

def DeepLabV3Plus_resnet50(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = Input(input_size)

    """ Pre-trained ResNet50 """
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    """ Pre-trained ResNet50 Output """
    image_features = base_model.get_layer('conv4_block6_out').output
    x_a = ASPP(image_features)
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)

    """ Get low-level features """
    x_b = base_model.get_layer('conv2_block2_out').output
    x_b = Conv2D(filters=48, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x = Concatenate()([x_a, x_b])

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((4, 4), interpolation="bilinear")(x)

    """ Outputs """
    x = Conv2D(n_classes, (1, 1), name='output_layer')(x)
    x = Activation('softmax')(x)

    """ Model """
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=metrics)
    model.summary()
    return model

def DeepLabV3Plus_resnet101(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = Input(input_size)

    """ Pre-trained ResNet1-1 """
    base_model = ResNet101(weights='imagenet', include_top=False, input_tensor=inputs)

    """ Pre-trained ResNet101 Output """
    image_features = base_model.get_layer('conv4_block6_out').output
    x_a = ASPP(image_features)
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)

    """ Get low-level features """
    x_b = base_model.get_layer('conv2_block2_out').output
    x_b = Conv2D(filters=48, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x = Concatenate()([x_a, x_b])

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((4, 4), interpolation="bilinear")(x)

    """ Outputs """
    x = Conv2D(n_classes, (1, 1), name='output_layer')(x)
    x = Activation('softmax')(x)

    """ Model """
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=[dice_acc])
    model.summary()
    return model


Model 2: DeepLabV3plusSE_EfficientNetB0


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
def SqueezeAndExcitation(inputs, ratio=8):

    b, h, w, c = inputs.shape
    x = GlobalAveragePooling2D()(inputs)
    x = Dense(c//ratio, activation='relu', use_bias=False)(x)
    x = Dense(c, activation='sigmoid', use_bias=False)(x)
    x = Multiply()([inputs, x])
    return x


def ASPP(image_features):
    shape = image_features.shape
    y_pool = AveragePooling2D(pool_size=(shape[1], shape[2]))(image_features)
    y_pool = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(y_pool)
    y_pool = BatchNormalization(name=f'bn_1')(y_pool)
    y_pool = Activation('relu', name=f'relu_1')(y_pool)
    y_pool = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y_pool)

    y_1 = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(image_features)
    y_1 = BatchNormalization(name=f'bn_2')(y_1)
    y_1 = Activation('relu', name=f'relu_2')(y_1)

    y_6 = Conv2D(filters=128, kernel_size=3, padding='same', dilation_rate = 6,use_bias=False)(image_features)
    y_6 = BatchNormalization(name=f'bn_3')(y_6)
    y_6 = Activation('relu', name=f'relu_3')(y_6)

    y_12 = Conv2D(filters=128, kernel_size=1, padding='same', dilation_rate = 12,use_bias=False)(image_features)
    y_12 = BatchNormalization(name=f'bn_4')(y_12)
    y_12 = Activation('relu', name=f'relu_4')(y_12)

    y_18 = Conv2D(filters=128, kernel_size=3, padding='same', dilation_rate = 6,use_bias=False)(image_features)
    y_18 = BatchNormalization(name=f'bn_5')(y_18)
    y_18 = Activation('relu', name=f'relu_5')(y_18)

    y_c = Concatenate()([y_pool, y_1, y_6, y_12, y_18])

    y = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(y_c)
    y = BatchNormalization(name=f'bn_6')(y)
    y = Activation('relu', name=f'relu_6')(y)
    return y


def DeepLabV3plusSE_EfficientNetB0(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = Input(input_size)

    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=inputs, drop_connect_rate=0.25)
    high_level_image_features = base_model.get_layer('block6a_expand_bn').output
    high_level_image_features = SqueezeAndExcitation(high_level_image_features, ratio=16)

    x_a = ASPP(high_level_image_features)
    x_a = SqueezeAndExcitation(x_a, ratio=16)
    x_a = UpSampling2D(size=4, interpolation='bilinear')(x_a)

    low_level_image_features = base_model.get_layer('block3a_expand_bn').output
    low_level_image_features = SqueezeAndExcitation(low_level_image_features, ratio=16)

    x_b = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(low_level_image_features)
    x_b = BatchNormalization(name=f'bn_7')(x_b)
    x_b = Activation('relu', name=f'relu_7')(x_b)

    x = Concatenate()([x_a, x_b])
    x  = SqueezeAndExcitation(x, ratio=16)

    x = Conv2D(filters=128, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization(name=f'bn_8')(x)
    x = Activation('relu', name=f'relu_8')(x)

    x = Conv2D(filters=128, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization(name=f'bn_9')(x)
    x = Activation('relu', name=f'relu_9')(x)

    x = UpSampling2D(size=4, interpolation='bilinear')(x)

    """ Outputs """
    x = Conv2D(n_classes, kernel_size=1, name='output_layer')(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=metrics)
    model.summary()
    return model


Model 3: DeepLabV3plus_DenseNet121

In [ ]:
from keras.applications.densenet import DenseNet121


def ASPP(image_features):

  shape = image_features.shape

  y_pool = AveragePooling2D(pool_size=(shape[1], shape[2]))(image_features)
  y_pool = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(y_pool)
  y_pool = BatchNormalization(name=f'bn_1')(y_pool)
  y_pool = Activation('relu', name=f'relu_1')(y_pool)
  y_pool = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y_pool)

  y_1 = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(image_features)
  y_1 = BatchNormalization(name=f'bn_2')(y_1)
  y_1 = Activation('relu', name=f'relu_2')(y_1)

  y_6 = Conv2D(filters=256, kernel_size=3, padding='same', dilation_rate = 6,use_bias=False)(image_features)
  y_6 = BatchNormalization(name=f'bn_3')(y_6)
  y_6 = Activation('relu', name=f'relu_3')(y_6)

  y_12 = Conv2D(filters=256, kernel_size=1, padding='same', dilation_rate = 12,use_bias=False)(image_features)
  y_12 = BatchNormalization(name=f'bn_4')(y_12)
  y_12 = Activation('relu', name=f'relu_4')(y_12)

  y_18 = Conv2D(filters=256, kernel_size=3, padding='same', dilation_rate = 6,use_bias=False)(image_features)
  y_18 = BatchNormalization(name=f'bn_5')(y_18)
  y_18 = Activation('relu', name=f'relu_5')(y_18)

  y_c = Concatenate()([y_pool, y_1, y_6, y_12, y_18])

  y = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(y_c)
  y = BatchNormalization(name=f'bn_6')(y)
  y = Activation('relu', name=f'relu_6')(y)

  return y


def DeepLabV3plus_DenseNet121(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = Input(input_size)


    base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=inputs)
    high_level_image_features = base_model.get_layer('pool4_relu').output

    x_a = ASPP(high_level_image_features)
    x_a = UpSampling2D(size=4, interpolation='bilinear')(x_a)

    low_level_image_features = base_model.get_layer('pool2_relu').output

    x_b = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(low_level_image_features)
    x_b = BatchNormalization(name=f'bn_7')(x_b)
    x_b = Activation('relu', name=f'relu_7')(x_b)

    x = Concatenate()([x_a, x_b])

    x = Conv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization(name=f'bn_8')(x)
    x = Activation('relu', name=f'relu_8')(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization(name=f'bn_9')(x)
    x = Activation('relu', name=f'relu_9')(x)

    x = UpSampling2D(size=4, interpolation='bilinear')(x)

    """ Outputs """
    x = Conv2D(n_classes, 1, name='output_layer')(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=metrics)
    model.summary()
    return model


Model: DeepLabV3plus_VGG16

In [ ]:
from keras.applications.vgg16 import VGG16


def ASPP(image_features):

    shape = image_features.shape

    y_pool = AveragePooling2D(pool_size=(shape[1], shape[2]))(image_features)
    y_pool = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(y_pool)
    y_pool = BatchNormalization(name=f'bn_1')(y_pool)
    y_pool = Activation('relu', name=f'relu_1')(y_pool)
    y_pool = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y_pool)

    y_1 = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(image_features)
    y_1 = BatchNormalization(name=f'bn_2')(y_1)
    y_1 = Activation('relu', name=f'relu_2')(y_1)

    y_6 = Conv2D(filters=128, kernel_size=3, padding='same', dilation_rate = 6,use_bias=False)(image_features)
    y_6 = BatchNormalization(name=f'bn_3')(y_6)
    y_6 = Activation('relu', name=f'relu_3')(y_6)

    y_12 = Conv2D(filters=128, kernel_size=1, padding='same', dilation_rate = 12,use_bias=False)(image_features)
    y_12 = BatchNormalization(name=f'bn_4')(y_12)
    y_12 = Activation('relu', name=f'relu_4')(y_12)

    y_18 = Conv2D(filters=128, kernel_size=3, padding='same', dilation_rate = 6,use_bias=False)(image_features)
    y_18 = BatchNormalization(name=f'bn_5')(y_18)
    y_18 = Activation('relu', name=f'relu_5')(y_18)

    y_c = Concatenate()([y_pool, y_1, y_6, y_12, y_18])

    y = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(y_c)
    y = BatchNormalization(name=f'bn_6')(y)
    y = Activation('relu', name=f'relu_6')(y)

    return y


def DeepLabV3plus_VGG16(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = Input(input_size)

    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=inputs)
    high_level_image_features = base_model.get_layer('block5_conv3').output

    x_a = ASPP(high_level_image_features)
    x_a = UpSampling2D(size=4, interpolation='bilinear')(x_a)

    low_level_image_features = base_model.get_layer('block3_conv3').output

    x_b = Conv2D(filters=128, kernel_size=1, padding='same', use_bias=False)(low_level_image_features)
    x_b = BatchNormalization(name=f'bn_7')(x_b)
    x_b = Activation('relu', name=f'relu_7')(x_b)

    x = Concatenate()([x_a, x_b])

    x = Conv2D(filters=128, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization(name=f'bn_8')(x)
    x = Activation('relu', name=f'relu_8')(x)

    x = Conv2D(filters=128, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization(name=f'bn_9')(x)
    x = Activation('relu', name=f'relu_9')(x)

    x = UpSampling2D(size=4, interpolation='bilinear')(x)

    """ Outputs """
    x = Conv2D(n_classes, 1, name='output_layer')(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=metrics)
    model.summary()
    return model

Model 7: Deeplabv3p_PSPNet_SqEx_SegModel

In [ ]:
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, Multiply
from tensorflow.keras.applications import EfficientNetB2


def SqueezeAndExcitation(inputs, ratio=8):

    b, h, w, c = inputs.shape

    x = GlobalAveragePooling2D()(inputs)
    x = Dense(c//ratio, activation='relu', use_bias=False)(x)
    x = Dense(c, activation='sigmoid', use_bias=False)(x)

    x = Multiply()([inputs, x])

    return x

def Pyramid_Pooling_Module(features, f=32, p1=2, p2=3, p3=8):

    shape = features.shape
    red = GlobalAveragePooling2D()(features)
    red = Reshape((1,1,shape[-1]))(red)
    red = Conv2D(filters=f, kernel_size=(1,1), padding='same', use_bias=False)(red)
    red = BatchNormalization()(red)
    red = Activation('relu')(red)
    red = UpSampling2D(size=shape[1],interpolation='bilinear')(red)

    orange = AveragePooling2D(pool_size=(p1))(features)
    orange = Conv2D(filters=f, kernel_size=(1,1), padding='same', use_bias=False)(orange)
    orange = BatchNormalization()(orange)
    orange = Activation('relu')(orange)
    orange = UpSampling2D(size=p1,interpolation='bilinear')(orange)

    blue = AveragePooling2D(pool_size=(p2))(features)
    blue = Conv2D(filters=f, kernel_size=(1,1), padding='same', use_bias=False)(blue)
    blue = BatchNormalization()(blue)
    blue = Activation('relu')(blue)
    blue = UpSampling2D(size=p2,interpolation='bilinear')(blue)

    green = AveragePooling2D(pool_size=(p3))(features)
    green = Conv2D(filters=f, kernel_size=(1,1), padding='same', use_bias=False)(green)
    green = BatchNormalization()(green)
    green = Activation('relu')(green)
    green = UpSampling2D(size=p3,interpolation='bilinear')(green)

    return Concatenate()([features, red, orange, blue, green])

def ASPP(image_features):

    shape = image_features.shape

    y_pool = AveragePooling2D(pool_size=(shape[1], shape[2]))(image_features)
    y_pool = Conv2D(filters=32, kernel_size=1, padding='same', use_bias=False)(y_pool)
    y_pool = BatchNormalization(name=f'bn_1')(y_pool)
    y_pool = Activation('relu', name=f'relu_1')(y_pool)
    y_pool = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y_pool)

    y_1 = Conv2D(filters=32, kernel_size=1, padding='same', use_bias=False)(image_features)
    y_1 = BatchNormalization(name=f'bn_2')(y_1)
    y_1 = Activation('relu', name=f'relu_2')(y_1)

    y_6 = Conv2D(filters=32, kernel_size=3, padding='same', dilation_rate = 6,use_bias=False)(image_features)
    y_6 = BatchNormalization(name=f'bn_3')(y_6)
    y_6 = Activation('relu', name=f'relu_3')(y_6)

    y_12 = Conv2D(filters=32, kernel_size=1, padding='same', dilation_rate = 12,use_bias=False)(image_features)
    y_12 = BatchNormalization(name=f'bn_4')(y_12)
    y_12 = Activation('relu', name=f'relu_4')(y_12)

    y_18 = Conv2D(filters=32, kernel_size=3, padding='same', dilation_rate = 6,use_bias=False)(image_features)
    y_18 = BatchNormalization(name=f'bn_5')(y_18)
    y_18 = Activation('relu', name=f'relu_5')(y_18)

    y_c = Concatenate()([y_pool, y_1, y_6, y_12, y_18])

    y = Conv2D(filters=32, kernel_size=1, padding='same', use_bias=False)(y_c)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    return y


def Deeplabv3p_PSPNet_SqEx_SegModel(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = Input(input_size)

    base_model = EfficientNetB2(weights='imagenet', include_top=False, input_tensor=inputs, drop_connect_rate=0.25)
    high_level_image_features = base_model.get_layer('block6a_expand_bn').output
    # high_level_image_features = SqueezeAndExcitation(high_level_image_features, ratio=16)

    x_a = ASPP(high_level_image_features)
    x_a = SqueezeAndExcitation(x_a, ratio=16)

    x_b = Pyramid_Pooling_Module(high_level_image_features, f=32, p1=2, p2=4, p3=8)
    x_b = Conv2D(filters=32, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)
    x_b = SqueezeAndExcitation(x_b, ratio=16)

    x_c = Concatenate()([x_a, x_b])
    x_c = Conv2D(filters=32, kernel_size=1, padding='same', use_bias=False)(x_c)
    x_c = BatchNormalization()(x_c)
    x_c = Activation('relu')(x_c)
    x_c = SqueezeAndExcitation(x_c, ratio=16)
    x_c = UpSampling2D(size=4, interpolation='bilinear')(x_c)

    low_level_image_features = base_model.get_layer('block3a_expand_bn').output
    x_d = Conv2D(filters=32, kernel_size=1, padding='same', use_bias=False)(low_level_image_features)
    x_d = BatchNormalization()(x_d)
    x_d = Activation('relu')(x_d)

    x_e = Concatenate()([x_c, x_d])
    x_e = Conv2D(filters=32, kernel_size=1, padding='same', use_bias=False)(x_e)
    x_e = BatchNormalization()(x_e)
    x_e = Activation('relu')(x_e)
    x_e = SqueezeAndExcitation(x_e, ratio=16)

    x = Conv2D(filters=32, kernel_size=3, padding='same', use_bias=False)(x_e)
    x = BatchNormalization(name=f'bn_8')(x)
    x = Activation('relu', name=f'relu_8')(x)

    x = Conv2D(filters=32, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization(name=f'bn_9')(x)
    x = Activation('relu', name=f'relu_9')(x)

    x = UpSampling2D(size=4, interpolation='bilinear')(x)

    """ Outputs """
    x = Conv2D(n_classes, 1, name='output_layer')(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=metrics)
    model.summary()
    return model

In [ ]:
SIZE_X = 256
SIZE_Y = 256
n_classes= 3
def get_model():
    return DeepLabV3plusSE_EfficientNetB0(n_classes=n_classes, IMG_HEIGHT=SIZE_Y, IMG_WIDTH=SIZE_X, IMG_CHANNELS=1)

model = get_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)
model.summary()

In [ ]:
import tensorflow as tf
strategy = tf.distribute.MultiWorkerMirroredStrategy()
with strategy.scope():
    model = DeepLabV3plusSE_EfficientNetB0()
#DeepLabV3Plus_resnet50
#DeepLabV3Plus_resnet101
#DeepLabV3plusSE_EfficientNetB0
#DeepLabV3plus_DenseNet121
#DeepLabV3plus_VGG16
#Deeplabv3p_PSPNet_SqEx_SegModel